In [4]:
## Import repositories
import os 
import numpy as np 
import pandas as pd
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
from utils import get_sample_freq, get_stimulus, get_test_freq

In [3]:
# Get stimulus dictionary 
degree_dict = get_stimulus()
print(degree_dict)

{0.0: array([ 17.641638,  57.666158, 132.712118, 172.736678, 202.755078,
       232.773518, 302.816518, 357.850318, 387.868758, 407.881078,
       492.733358, 527.754878, 542.764118, 607.804078, 647.828718,
       707.865598]), 20.0: array([ 67.672278,  77.678398, 102.693718, 177.739758, 227.770438,
       247.782718, 287.807318, 307.819598, 392.871838, 432.896438,
       467.717958, 512.745638, 582.788718, 612.807158, 637.822558,
       677.847158]), 40.0: array([ 47.660038,  82.681478, 107.696798, 112.699878, 237.776598,
       262.791958, 317.825758, 327.831878, 362.853398, 412.884158,
       457.711798, 522.751798, 552.770238, 627.816398, 697.859478,
       717.871758]), 60.0: array([  2.632478,  27.647758,  97.690678, 147.721318, 197.752038,
       267.794998, 297.813438, 352.847238, 367.856478, 372.859558,
       472.721038, 537.761038, 587.791798, 592.794878, 662.837918,
       692.856398]), 80.0: array([ 72.675358,  87.684518, 122.705998, 152.724398, 252.785798,
       257.7888

In [ ]:
os.makedirs('./sample/images', exist_ok=True)
os.makedirs('./test/images', exist_ok=True)

#Save neuron (sample) data to csv format 
for i in range(1, 11):
    df = get_sample_freq(i)
    df.to_csv(f'./sample/sample_spike_neuron{i}.csv', index = False)

#Save case (test) data to csv format 
for i in range(1, 11):
    df = get_test_freq(i)
    df.to_csv(f'./test/test_spike_case{i}.csv', index = False)



In [ ]:
# Save barplots 
for sample_num in range(1,11):
    plt.figure(figsize=(5,5), dpi = 150)
    sns.barplot(data = pd.read_csv(f'./sample/sample_spike_neuron{sample_num}.csv'), x = 'Degree', y = 'Frequency').set(title= f'Neuron {sample_num}')
    plt.tight_layout()
    plt.savefig(f'./sample/images/sample_spike_neuron{sample_num}.png')
    plt.close()

for num in range(1, 11):
    plt.figure(figsize = (5,5), dpi = 150)
    sns.barplot(data = pd.read_csv(f'./test/test_spike_case{num}.csv'), x = 'Degree', y = 'Frequency', color='orange').set(title = f'Case {num}')
    plt.tight_layout()
    plt.savefig(f'./test/images/test_spike_case{num}.png')
    plt.close()

In [ ]:
stimulus_raw = scipy.io.loadmat('dat_stimulus.mat')
spk_sample_raw = scipy.io.loadmat('dat_spk_sample.mat')
spk_test_raw = scipy.io.loadmat('dat_spk_test.mat')

stimulus = stimulus_raw['stimulus']
spk_sample = spk_sample_raw['spk_sample']
spk_test = spk_test_raw['spk_test']

sample_num = 9


start_gap = 0.1
window_size = 2
end_time = start_gap + window_size
sample = spk_sample[sample_num-1,0]

#index = 
for key in degree_dict:
    print(key)
    stim_list = degree_dict[key]
    assert len(stim_list) == 16
    frequency_list = []
    for stim_time in stim_list:
        if (stim_time >= 50) & (stim_time <= 350):
            frequency = len(sample[(sample > (stim_time+start_gap)) & (sample <= (stim_time + end_time))]) / window_size
            frequency_list.append(frequency)
        #else:
        #    continue
    print(frequency_list)
    print(np.round(np.mean(frequency_list), 3), np.round(np.std(frequency_list), 3))

In [ ]:
#Statistical test 
from scipy.stats import ttest_ind

significance_level = 0.1
for test_num in range(1, 11):
    test_spike = pd.read_csv(f'./test/test_spike_case{test_num}.csv')
    for i in range(1, 11):
        sample_spike = pd.read_csv(f'./sample/sample_spike_neuron{i}.csv')
        p_list = []
        for degree in np.unique(stimulus[:,1]):
            test = test_spike[test_spike['Degree'] == int(degree)]['Frequency']
            sample = sample_spike[sample_spike['Degree'] == int(degree)]['Frequency']
            _, p = ttest_ind(test, sample)
            p_list.append(p >= significance_level)
        #print(sum(p_list))
        if sum(p_list) > 7:
        #if all(p_list):
            print(f'Case {test_num} matches with Neuron {i}')

In [ ]:

from scipy.stats import entropy

def to_density(spike):
    pdf = []
    stimulus = scipy.io.loadmat('dat_stimulus.mat')['stimulus']
    for degree in np.unique(stimulus[:,1]):
        pdf.append(spike[spike['Degree'] == int(degree)]['Frequency'].mean() + 1e-6)
    
    pdf = np.asarray(pdf) 
    pdf /= (np.sum(pdf) + len(pdf)*1e-6)
    return pdf

for test_num in range(1, 11):
    test_spike = pd.read_csv(f'./test/test_spike_case{test_num}.csv')
    test_pdf = to_density(test_spike)

    KLdiv_list = []
    for i in range(1, 11):
        sample_spike = pd.read_csv(f'./sample/sample_spike_neuron{i}.csv')
        sample_pdf = to_density(sample_spike)
        KLdiv = entropy(pk=sample_pdf, qk = test_pdf)
        KLdiv_list.append(KLdiv)
    print(f'Case {test_num} matches with Neuron {np.argmin(KLdiv_list) + 1}') 